In [ ]:
# ===================================================================
#           数据指挥中心仪表盘 (Data QA Dashboard) v4.1
# ===================================================================
#
# 新增功能: 集成 itables，提供分页、搜索、排序的交互式表格显示。
#
# -------------------------------------------------------------------

# 1. 导入必要的库
import pandas as pd
import yaml
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output



# ---> 核心：导入并初始化 itables <---
from itables import init_notebook_mode, show
# 设置 itables 为全局交互模式，所有DataFrame输出都会升级
init_notebook_mode(all_interactive=True)
# 你可以自定义一些选项，比如每页显示的数量
import itables.options as opt
opt.lengthMenu = [10, 20, 50, 100]
opt.classes = ["display", "compact"] # 使用更紧凑的显示风格

# 2. 加载配置
try:
    with open('config.yaml', 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    BASE_PATH = Path(config['storage']['base_path'])
    print(f"✅ 成功加载配置，数据根目录: {BASE_PATH.resolve()}")
except FileNotFoundError:
    print("❌ 未找到 config.yaml 文件。")
    BASE_PATH = Path("data")

# --- 3. 核心功能：交互式数据浏览器 ---
data_map = {}
if BASE_PATH.exists():
    for data_type_dir in BASE_PATH.iterdir():
        if data_type_dir.is_dir():
            data_type = data_type_dir.name
            entities = [entity_dir.name.split('=')[1] for entity_dir in data_type_dir.glob("entity=*")]
            if entities:
                data_map[data_type] = sorted(entities)

data_type_dropdown = widgets.Dropdown(options=[''] + list(data_map.keys()), description='数据类型:')
entity_dropdown = widgets.Dropdown(description='选择文件:', disabled=True)
output_area = widgets.Output()

def on_data_type_change(change):
    selected_type = change.new
    if selected_type in data_map:
        entity_dropdown.options = [''] + data_map[selected_type]
        entity_dropdown.disabled = False
    else:
        entity_dropdown.options = []
        entity_dropdown.disabled = True

def on_entity_change(change):
    with output_area:
        clear_output(wait=True)
        selected_type = data_type_dropdown.value
        selected_entity = change.new
        if not selected_type or not selected_entity:
            return

        file_path = BASE_PATH / selected_type / f"entity={selected_entity}" / "data.parquet"
        print(f"正在读取文件: {file_path}")
        try:
            df = pd.read_parquet(file_path)
            print(f"文件加载成功，共 {len(df)} 行 x {len(df.columns)} 列。")
            
            # ---> 核心：现在我们直接 display(df) <---
            # itables 会自动将其转换为一个功能强大的交互式表格
            display(df.sort_index(ascending=False))
            
        except Exception as e:
            print(f"❌ 读取或显示文件时出错: {e}")

data_type_dropdown.observe(on_data_type_change, names='value')
entity_dropdown.observe(on_entity_change, names='value')

print("\n" + "="*20, "交互式数据浏览器 (Powered by itables)", "="*20)
print("请从下拉菜单中选择您想查看的数据文件：")
ui = widgets.VBox([data_type_dropdown, entity_dropdown, output_area])
display(ui)